In [ ]:
# Code by Melinda Kleczynski 
# Data from Christina Bergonzo 

# Finalized March 12, 2025

# Read a csv file with 3d spatial coordinates 
# Return concatenated 0d, 1d, and 2d normalized Gaussian Betti curves 

# Change frame_data_fpath as needed based on file structure of input data 
# Change betti012_fpath as needed based on desired file structure of output data 

In [ ]:
import numpy as np 
import oat_python as oat
import pandas as pd 

from gccd import smooth_betti 

In [ ]:
# parameter selection 

first_eps = 1 
last_eps = 30 
epsilons = np.linspace(first_eps, last_eps, 1+10*(last_eps-first_eps))

sigma_exponents = [s for s in range(1, 7)]

In [ ]:
def smooth_normalized_curve(eps_vec, ph_df, k_012, sigma_float):

    ph_k = ph_df[ph_df.dimension == k_012]
    births_k = np.array(ph_k.birth)
    deaths_k = np.array(ph_k.death)
    nbars_k = len(births_k)

    smooth_betti_k = [smooth_betti(epsilon, births_k, deaths_k, sigma_float) for epsilon in eps_vec]
    return np.array(smooth_betti_k)/nbars_k 

In [ ]:
dataset_name = 'Fc_glycans'  # 'Fc_glycans' or 'Fc_noglycans' 
trajectory_num = 0  # 0, 1, 2, or 3  

for frame_num in range(200, 1000):  

    # get coordinate data
    frame_data_fpath = ('formatted_data\\' + dataset_name + 
                        '\\traj' + str(trajectory_num) + 
                        '\\' + dataset_name + 
                        '_traj' + str(trajectory_num) + 
                        '_frame' + str(frame_num) + '.csv')
    frame_data = pd.read_csv(frame_data_fpath)
    atom_coords = np.array(frame_data[['x', 'y', 'z']])

    # perform TDA 
    maxrad = oat.dissimilarity.enclosing_from_cloud(atom_coords) + 0.001 
    dissimilairty_matrix = oat.dissimilarity.matrix_from_cloud(cloud = atom_coords, 
                                                               dissimilarity_max = maxrad)
    boundary = oat.rust.FactoredBoundaryMatrixVr(dissimilarity_matrix = dissimilairty_matrix, 
                                                 homology_dimension_max = 2)
    ph = boundary.homology(return_cycle_representatives = False, return_bounding_chains = False)

    # create and save curves 
    for sigma_exp in sigma_exponents:
        sigma = (1/2)**sigma_exp
        concat_curves = np.hstack([smooth_normalized_curve(epsilons, ph, k, sigma) for k in range(3)])
        betti012_fpath = ('betti012s\\' + dataset_name + '\\traj' + str(trajectory_num) + 
                       '\\' + dataset_name + '_traj' + str(trajectory_num) + 
                       '_frame' + str(frame_num) + '_sigmaexp' + str(sigma_exp) + '_betti012.csv')
        pd.DataFrame(concat_curves).to_csv(betti012_fpath, index = False)